## [--- Day 12: Secret Entrance ---](https://adventofcode.com/2025/day/12)

In [ ]:
import re
from dataclasses import dataclass


@dataclass
class Grid:
    width: int
    height: int
    cells: int
    requirements: tuple[int, int, int, int, int, int]
    # Bitmasks for flood fill
    left_mask: int
    """bitmask representing the left edge of the grid"""
    right_mask: int
    """bitmask representing the right edge of the grid"""
    full_mask: int
    """bitmask representing the full grid"""
    current: int = 0
    """bitmask representing the current state of the grid"""


def build_grids(data: list[re.Match]) -> list[Grid]:
    grids: list[Grid] = []
    for _, dimensions, reqs in data:
        shape_requirements = tuple(map(int, reqs.split()))
        if len(shape_requirements) != 6:
            raise ValueError("Each requirement must have exactly 6 integers.")
        x, y = map(int, dimensions.split("x"))
        grids.append(
            Grid(
                width=x,
                height=y,
                cells=x * y,
                requirements=shape_requirements[0:6],
                left_mask=sum(1 << (r * x) for r in range(y)),
                right_mask=sum(1 << (r * x + (x - 1)) for r in range(y)),
                full_mask=(1 << (x * y)) - 1,
            )
        )

    return grids


def build_shapes(raw_shapes: list[re.Match]) -> list[set[tuple[int, int, int]]]:
    all_shapes: list[set[tuple[int, int, int]]] = []
    for match in raw_shapes:
        raw_shape = parse_shape(match[0].splitlines())
        shape_set: set[tuple[int, int, int]] = set([raw_shape, reflect(raw_shape)])

        # Generate all rotations and reflections
        for _ in (1, 2, 3):
            raw_shape = rotate(raw_shape)
            shape_set.add(raw_shape)
            shape_set.add(reflect(raw_shape))

        all_shapes.append(shape_set)

    return all_shapes


def parse_shape(shape: list[str]) -> tuple[int, int, int]:
    return (
        1 * (shape[0][0] == "#") | 2 * (shape[0][1] == "#") | 4 * (shape[0][2] == "#"),
        1 * (shape[1][0] == "#") | 2 * (shape[1][1] == "#") | 4 * (shape[1][2] == "#"),
        1 * (shape[2][0] == "#") | 2 * (shape[2][1] == "#") | 4 * (shape[2][2] == "#"),
    )


def rotate(shape: tuple[int, int, int]) -> tuple[int, int, int]:
    return (
        (((shape[0] & 1) << 2) | ((shape[1] & 1) << 1) | (shape[2] & 1)),
        (((shape[0] & 2) << 1) | (shape[1] & 2) | ((shape[2] & 2) >> 1)),
        ((shape[0] & 4) | ((shape[1] & 4) >> 1) | ((shape[2] & 4) >> 2)),
    )


def reflect(shape: tuple[int, int, int]) -> tuple[int, int, int]:
    return (
        ((shape[0] & 1) << 2) | (shape[0] & 2) | ((shape[0] & 4) >> 2),
        ((shape[1] & 1) << 2) | (shape[1] & 2) | ((shape[1] & 4) >> 2),
        ((shape[2] & 1) << 2) | (shape[2] & 2) | ((shape[2] & 4) >> 2),
    )


def print_shape(shape: tuple[int, int, int]) -> None:
    grid = (
        ("# " if (shape[y] & x) != 0 else ". " for x in (1, 2, 4)) for y in (0, 1, 2)
    )

    print("\n".join("".join(row) for row in grid))
    print("-" * 5)


def create_placements(
    grid: Grid, shapes: list[set[tuple[int, int, int]]]
) -> list[list[list[int]]]:
    """
    Generates all valid grid masks for shape variants, indexed by their 'anchor bit'
    (the lowest set bit in the mask).

    Since every shape has a 3x3 footprint with no empty rows, the anchor is
    guaranteed to be in the first row.

    This enables a 'First Empty Bit' search strategy: the solver finds the first
    hole in the grid and only checks masks whose anchor matches that specific bit.
    """
    W, H = grid.width, grid.height
    placements: list[list[list[int]]] = [[[] for _ in range(6)] for _ in range(W * H)]

    for shape_idx, variants in enumerate(shapes):
        for shape in variants:
            # lowest set bit for this shape
            for y in range(H - 2):
                y0_W = y * W
                y1_W = (y + 1) * W
                y2_W = (y + 2) * W
                for x in range(W - 2):
                    mask = (
                        shape[0] << (x + y0_W)
                        | shape[1] << (x + y1_W)
                        | shape[2] << (x + y2_W)
                    )
                    anchor = (mask & -mask).bit_length() - 1
                    placements[anchor][shape_idx].append(mask)
    return placements


def get_island(grid: Grid, target_bit: int) -> int:
    empty_space = ~grid.current & grid.full_mask
    flood = 1 << target_bit

    while True:
        next_flood = (
            flood
            | (
                ((flood << 1) & ~grid.right_mask)  # left
                | ((flood >> 1) & ~grid.left_mask)  # right
                | (flood << grid.width)  # down
                | (flood >> grid.width)  # up
            )
            & empty_space
        )

        if next_flood == flood:
            return flood
        flood = next_flood


def solve_grid(
    grid: Grid,
    requirements: list[int],
    placements: list[list[list[int]]],
    area_needed: int,
    shape_weights: tuple[int, ...],
) -> bool:
    current = grid.current
    # Base cases
    if area_needed == 0 or not any(requirements):  # all shapes placed
        return True

    # Prune if the remaining area cannot fit all remaining shapes
    if area_needed > grid.cells - current.bit_count():  # not enough space left
        return False

    # Prune if target bit has exhausted the grid
    target_bit = (current ^ (current + 1)).bit_length() - 1
    if target_bit >= grid.cells:
        return False

    # Fill the gap if this local area is too small for any remaining shape
    island = get_island(grid, target_bit)
    min_shape_weight = min(shape_weights[i] for i in range(6) if requirements[i] > 0)
    if island.bit_count() < min_shape_weight:
        grid.current = current | island
        is_solved = solve_grid(
            grid,
            requirements,
            placements,
            area_needed,
            shape_weights,
        )
        grid.current = current
        return is_solved

    for shape_id in range(6):
        if requirements[shape_id] == 0:
            continue
        for mask in placements[target_bit][shape_id]:
            if not (current & mask):
                grid.current = current | mask
                requirements[shape_id] -= 1
                if solve_grid(
                    grid,
                    requirements,
                    placements,
                    area_needed - shape_weights[shape_id],
                    shape_weights,
                ):
                    return True
                requirements[shape_id] += 1
                grid.current = current
    return False
    # grid.current = current | (1 << target_bit)
    # if solve_grid(
    #     grid,
    #     requirements,
    #     placements,
    #     area_needed,
    #     shape_weights,
    # ):
    #     return True
    # else:
    #     grid.current = current
    #     return False


def solve() -> None:
    with open("..\\data\\12.txt") as file:
        pattern = re.compile(r"^([#.\n]{11})|(\d+x\d+): ((?:\d+ ?){6})$", re.MULTILINE)
        data = pattern.findall(file.read())

    raw_shapes = data[:6]
    shapes: list[set[tuple[int, int, int]]] = build_shapes(raw_shapes)
    shape_weights: tuple[int, ...] = tuple(
        sum(row.bit_count() for row in list(s)[0]) for s in shapes
    )
    grids: list[Grid] = build_grids(data[6:])
    results: list[tuple[Grid, bool]] = []

    for grid in grids:
        max_shape_area = sum(9 * grid.requirements[i] for i in range(6))
        if max_shape_area <= grid.cells:  # trivially solvable
            results.append((grid, True))
            continue
        placements = create_placements(grid, shapes)
        is_solvable = solve_grid(
            grid,
            list(grid.requirements),
            placements,
            sum(shape_weights[i] * grid.requirements[i] for i in range(6)),
            shape_weights,
        )
        results.append((grid, is_solvable))
        print(f"Tested grid {grid.width}x{grid.height}: {is_solvable}")

    count = 0
    for grid, solvable in results:
        if solvable:
            count += 1
    print(f"Total solvable grids: {count}/{len(grids)}")


solve()

Tested grid 49x48: False
Tested grid 48x41: False
Tested grid 36x43: False
Tested grid 49x35: False
Tested grid 48x35: False
Tested grid 40x45: False
Tested grid 44x37: False
Tested grid 41x38: False
Tested grid 36x36: False
Tested grid 42x38: False
Tested grid 38x40: False
Tested grid 48x46: False
Tested grid 38x42: False
Tested grid 42x42: False
Tested grid 42x48: False
Tested grid 43x45: False
Tested grid 48x49: False
Tested grid 44x36: False
Tested grid 35x42: False
Tested grid 45x40: False
Tested grid 35x48: False
Tested grid 35x46: False
Tested grid 46x37: False
Tested grid 40x50: False
Tested grid 48x43: False
Tested grid 43x37: False
Tested grid 49x49: False
Tested grid 45x43: False
Tested grid 45x38: False
Tested grid 47x50: False
Tested grid 47x47: False
Tested grid 38x41: False
Tested grid 37x43: False
Tested grid 44x47: False
Tested grid 46x40: False
Tested grid 41x40: False
Tested grid 46x41: False
Tested grid 48x47: False
Tested grid 36x49: False
Tested grid 42x48: False


In [46]:
shape1: list[str] = ["# # # ", "# # . ", "# # . "]
shape1_v = [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (0, 2), (1, 2)]
shape1_r = rotate(shape1_v)
reconstructed = [[". "] * 3 for _ in range(3)]
for x, y in shape1_r:
    reconstructed[y][x] = "# "


shape1_expected = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (2, 0), (2, 1)]
expected = [[". "] * 3 for _ in range(3)]
for x, y in shape1_expected:
    expected[y][x] = "# "

print("start:")
print("\n".join(shape1))
print("rotated:")
print("\n".join("".join(row) for row in reconstructed))
print("expected:")
print("\n".join("".join(row) for row in expected))


TypeError: unsupported operand type(s) for &: 'tuple' and 'int'

In [ ]:
from timeit import timeit

shape2: list[str] = ["###", "##.", "##."]


def parse_shape_1(shape: list[str]) -> tuple[int, int, int]:
    return (
        1 * (shape[0][0] == "#") | 2 * (shape[0][1] == "#") | 4 * (shape[0][2] == "#"),
        1 * (shape[1][0] == "#") | 2 * (shape[1][1] == "#") | 4 * (shape[1][2] == "#"),
        1 * (shape[2][0] == "#") | 2 * (shape[2][1] == "#") | 4 * (shape[2][2] == "#"),
    )


print(parse_shape_1(shape2))


def rotate_1(shape: tuple[int, int, int]) -> tuple[int, int, int]:
    return (
        (((shape[0] & 1) << 2) | ((shape[1] & 1) << 1) | (shape[2] & 1)),
        (((shape[0] & 2) << 1) | (shape[1] & 2) | ((shape[2] & 2) >> 1)),
        ((shape[0] & 4) | ((shape[1] & 4) >> 1) | ((shape[2] & 4) >> 2)),
    )


print(rotate_1(shape_ints))  # [7, 7, 4]


def reflect_1(shape: tuple[int, int, int]) -> tuple[int, int, int]:
    return (
        ((shape[0] & 1) << 2) | (shape[0] & 2) | ((shape[0] & 4) >> 2),
        ((shape[1] & 1) << 2) | (shape[1] & 2) | ((shape[1] & 4) >> 2),
        ((shape[2] & 1) << 2) | (shape[2] & 2) | ((shape[2] & 4) >> 2),
    )


print(reflect(shape_ints))  # (7, 6, 6)


def print_shape_1(shape: tuple[int, int, int]) -> None:
    grid = (
        ("# " if (shape[y] & x) != 0 else ". " for x in (1, 2, 4)) for y in (0, 1, 2)
    )

    print("\n".join("".join(row) for row in grid))
    print("-" * 6)


print_shape_1(shape_ints)  # prints the shape
print_shape_1(rotate_1(shape_ints))  # prints the rotated shape
print_shape_1(reflect_1(rotate_1(shape_ints)))  # prints the reflected shape

print(timeit(lambda: print_shape_1(shape_ints), number=1000000))
print(timeit(lambda: print_shape_2(shape_ints), number=1000000))


(7, 3, 3)
(7, 7, 4)
[7, 6, 6]
1.1171965999528766
0.2825727998279035


I'm working on AoC-2025 problems to develop my programming skills. Here is the problem I am currently working on:

--- Day 12: Christmas Tree Farm ---
You're almost out of time, but there can't be much left to decorate. Although there are no stairs, elevators, escalators, tunnels, chutes, teleporters, firepoles, or conduits here that would take you deeper into the North Pole base, there is a ventilation duct. You jump in.
After bumping around for a few minutes, you emerge into a large, well-lit cavern full of Christmas trees!
There are a few Elves here frantically decorating before the deadline. They think they'll be able to finish most of the work, but the one thing they're worried about is the presents for all the young Elves that live here at the North Pole. It's an ancient tradition to put the presents under the trees, but the Elves are worried they won't fit.
The presents come in a few standard but very weird shapes. The shapes and the regions into which they need to fit are all measured in standard units. To be aesthetically pleasing, the presents need to be placed into the regions in a way that follows a standardized two-dimensional unit grid; you also can't stack presents.
As always, the Elves have a summary of the situation (your puzzle input) for you. First, it contains a list of the presents' shapes. Second, it contains the size of the region under each tree and a list of the number of presents of each shape that need to fit into that region. For example:

0:
###
##.
##.

1:
###
##.
.##

2:
.##
###
##.

3:
##.
###
##.

4:
###
#..
###

5:
###
.#.
###

4x4: 0 0 0 0 2 0
12x5: 1 0 1 0 2 2
12x5: 1 0 1 0 3 2
The first section lists the standard present shapes. For convenience, each shape starts with its index and a colon; then, the shape is displayed visually, where # is part of the shape and . is not.
The second section lists the regions under the trees. Each line starts with the width and length of the region; 12x5 means the region is 12 units wide and 5 units long. The rest of the line describes the presents that need to fit into that region by listing the quantity of each shape of present; 1 0 1 0 3 2 means you need to fit one present with shape index 0, no presents with shape index 1, one present with shape index 2, no presents with shape index 3, three presents with shape index 4, and two presents with shape index 5.
Presents can be rotated and flipped as necessary to make them fit in the available space, but they have to always be placed perfectly on the grid. Shapes can't overlap (that is, the # part from two different presents can't go in the same place on the grid), but they can fit together (that is, the . part in a present's shape's diagram does not block another present from occupying that space on the grid).
The Elves need to know how many of the regions can fit the presents listed. In the above example, there are six unique present shapes and three regions that need checking.
The first region is 4x4:

....
....
....
....
In it, you need to determine whether you could fit two presents that have shape index 4:

###
#..
###
After some experimentation, it turns out that you can fit both presents in this region. Here is one way to do it, using A to represent one present and B to represent the other:

AAA.
ABAB
ABAB
.BBB
The second region, 12x5: 1 0 1 0 2 2, is 12 units wide and 5 units long. In that region, you need to try to fit one present with shape index 0, one present with shape index 2, two presents with shape index 4, and two presents with shape index 5.
It turns out that these presents can all fit in this region. Here is one way to do it, again using different capital letters to represent all the required presents:

....AAAFFE.E
.BBBAAFFFEEE
DDDBAAFFCECE
DBBB....CCC.
DDD.....C.C.
The third region, 12x5: 1 0 1 0 3 2, is the same size as the previous region; the only difference is that this region needs to fit one additional present with shape index 4. Unfortunately, no matter how hard you try, there is no way to fit all of the presents into this region.
So, in this example, 2 regions can fit all of their listed presents.
Consider the regions beneath each tree and the presents the Elves would like to fit into each of them. How many of the regions can fit all of the presents listed?

I've never tried to solve a problem like this before. I want to optimize for execution time and maintain best practices. The grid sizes approach 50x50, there are six shapes with a strict 3x3 footprint (no 3x1). Just like the example, all three rows and columns have bits, but there are gaps. The shapes are parsed into tuple[int, int, int] where the LSB is left (reveresed). With this structure, I can create "placements" for every possible postion within the grid's width, indexing them into a list by their first active bit. Carefully creating the placements will eliminate the need for bounds checks, we won't create such invalid placements.
I'm considering:
1. Use integers to represent the grid and where shapes could sit so I can use bitmasking.
2. parse shapes into integers for each row
3. create sets of each shape's rotations and reflections.
4. convert each grid into an integer.
5. convert shape coordinates into grid offsets.
6. calculate shape placements for each grid's dimensions.
7. recursively search each grid and backtrack to prune the search space.
   1. Create list of placements, per shape, as an int for this grid, where the index is the anchor bit.
      1. Proposed data structure is a cache of nested lists `width cache` → `shapes` → `placements (by first active bit)`
      2. The loops are hot, so a reference to `shapes` is pulled from the width cache when a new grid starts.
   2. bitwise floodfill to find the size of holes.
   3. skip holes that are too small for a given shape.
   4. exit if the remaining space is less than the minimum space required for all remaining shapes.

Please guide me through solving this problem in Python, but do not provide any code unless I ask for it. The solution solves the first two example problems quickly, but the unsolvable 12x5 is taking far too long. I think the island jumps are working but when backtracking, the skip bit solve check still runs. Review the code and look for inefficiencies that are blowing out the run time. Here's my draft code:

In [ ]:
import re
import os
from functools import lru_cache


def get_variants(coords):
    vars = set()
    curr = coords
    for _ in range(4):
        for s in [curr, tuple((r, -c) for r, c in curr)]:
            min_r, min_c = min(r for r, c in s), min(c for r, c in s)
            vars.add(tuple(sorted((r - min_r, c - min_c) for r, c in s)))
        curr = tuple((c, -r) for r, c in curr)
    return list(vars)


def solve():
    file_path = "..\\data\\12 example.txt"
    with open(file_path, "r") as f:
        content = f.read()

    # --- 1. Regex Extraction ---
    # Extract shapes
    shape_raw = re.findall(r"\d+:\n([#.\n]+?)(?=\n\n|\d+x|$)", content, re.S)
    shapes = []
    for s in shape_raw:
        coords = []
        for r, line in enumerate(s.strip().splitlines()):
            for c, char in enumerate(line):
                if char == "#":
                    coords.append((r, c))
        shapes.append(get_variants(tuple(coords)))

    # Extract regions
    region_data = re.findall(r"(\d+)x(\d+): ([\d ]+)", content)

    # --- 2. High-Performance Solver ---
    def solve_region(w, h, counts):
        # Flatten counts into a list of shape indices
        needed = []
        for s_idx, qty in enumerate(counts):
            needed.extend([s_idx] * qty)

        # Heuristic: Sort by Area (Descending) to fail fast
        needed.sort(key=lambda x: len(shapes[x][0]), reverse=True)
        needed = tuple(needed)

        # Pre-calculate masks for the specific grid dimensions
        memo_masks = {}
        for s_idx in set(needed):
            s_masks = []
            for var in shapes[s_idx]:
                v_h = max(r for r, c in var) + 1
                v_w = max(c for r, c in var) + 1
                for r in range(h - v_h + 1):
                    for c in range(w - v_w + 1):
                        m = sum(1 << ((r + dr) * w + (c + dc)) for dr, dc in var)
                        s_masks.append(m)
            memo_masks[s_idx] = tuple(s_masks)

        @lru_cache(None)
        def backtrack(grid_mask, shape_idx):
            if shape_idx == len(needed):
                return True

            s_idx = needed[shape_idx]

            # Optimization: Symmetry Breaking for identical shapes
            # If the next shape is the same as the current one,
            # we don't need to re-scan placements we've already tried.
            for m in memo_masks[s_idx]:
                if not (grid_mask & m):
                    if backtrack(grid_mask | m, shape_idx + 1):
                        return True
            return False

        return backtrack(0, 0)

    # --- 3. Execution ---
    fits = 0
    for w_str, h_str, counts_str in region_data:
        w, h = int(w_str), int(h_str)
        counts = list(map(int, counts_str.split()))
        if solve_region(w, h, tuple(counts)):
            fits += 1
        print(
            f"Region {w}x{h} with counts {counts} fits: {solve_region(w, h, tuple(counts))}"
        )

    print(f"Number of regions that fit: {fits}")


if __name__ == "__main__":
    solve()

Region 4x4 with counts [0, 0, 0, 0, 2, 0] fits: True
Region 12x5 with counts [1, 0, 1, 0, 2, 2] fits: True
Region 12x5 with counts [1, 0, 1, 0, 3, 2] fits: False
Number of regions that fit: 2
